In [3]:
from transformers import ViTFeatureExtractor, ViTModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler

from PIL import Image
import numpy as np
import glob
import os
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef
from sklearn.model_selection import train_test_split

In [2]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224') 

In [10]:
feature_extractor(Image.open('data/asl_alphabet_train/asl_alphabet_train/A/A1.jpg'))['pixel_values'][0].max()

1.0